<a href="https://colab.research.google.com/github/tsubauaaa/Tensorflow2.0-tutorials/blob/master/Tensorflow2_text_classfication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TensorFlow Hub: Movie reviewsを使ったテキスト分類
このノートブックは、レビューのテキストを使用して、映画レビューをポジティブまたはネガティブに分類する。これは、バイナリまたは2クラスの分類の例であり、重要かつ広く適用可能な種類の機械学習問題。

このチュートリアルでは、TensorFlow HubとKerasを使用した転移学習の基本的なアプリケーションを示す。

インターネットムービーデータベースの50,000件の映画レビューのテキストを含むIMDBデータセットを使用する。これらは、トレーニング用の25,000件のレビューとテスト用の25,000件のレビューに分かれています。トレーニングとテストのセットはバランスが取れており、同数の肯定的レビューと否定的レビューが含まれている。

このノートブックは、TensorFlowでモデルを構築およびトレーニングするための高レベルAPIであるtf.kerasと、転移学習用のライブラリおよびプラットフォームであるTensorFlow Hubを使用する。 

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
!pip install --upgrade tensorflow-gpu
import numpy as np

import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")


     |████████████████████████████████| 380.8MB 77kB/s 
     |████████████████████████████████| 450kB 39.7MB/s 
     |████████████████████████████████| 3.8MB 32.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=e77d6795e2636d906871fabb7f34cfd8fc2cb505a0b6c1fdbcc21699b24f5f0a
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 1.14.0 has requirement tensorboard<1.15.0,>=1.14.0, but you'll have tensorboard 2.0.0 which is incompatible.
ERROR: tensorflow 1.14.0 has requirement tensorflow-estimator<1.15.0rc0,>=1.14.0rc0, but you'll have tensorflow-estimator 2.0.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tensorflow-estimator-1.14.0
  Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninsta

### ダウンロード IMDB データセット

In [0]:
# トレーニングデータを60%と40%で分割するとトレーニング15000、検証10000、テスト25000になる
train_validation_split = tfds.Split.TRAIN.subsplit([6, 4])
(train_data, validation_data), test_data = tfds.load(
    name="imdb_reviews",
    split=(train_validation_split, tfds.Split.TEST),
    as_supervised=True
)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


### データの探索
各行は、映画のレビューと対応するラベルを表す。ラベルは0または1の整数値で、0は否定的なレビュー、1は肯定的なレビュー。10行出してみる。

In [0]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch
train_labels_batch

<tf.Tensor: id=220, shape=(10,), dtype=int64, numpy=array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0])>

### モデルのビルド
ニューラルネットワークはレイヤーを積み重ねて作成される。これには以下の3つの主要なアーキテクチャが必要。  
* テキスト表現
* 使用するレイヤーの数
* 各レイヤーで使用するユニット数  

入力データはレビュー文で予測するのはラベル(0 or 1)

テキストを表す1つの方法は、文を埋め込みベクトルに変換する。  
事前トレーニング済みのテキスト埋め込みを最初のレイヤーとして使用する。  
これには次の3つの利点がある。  
* テキストの前処理を心配する必要がない
* 転移学習の恩恵を受けられる
* 埋め込みのサイズが固定されているため、処理が簡単  

このチュートリアルでは、google/tf2-preview/gnews-swivel-20dim/1というTensorFlow Hubの事前学習済みテキスト埋め込みモデルを使用する。

まず、TensorFlow Hubモデルを使用して文を埋め込むKerasレイヤーを作成し、いくつかの入力例で試してみる。入力テキストの長さに関係なく、埋め込みの出力形状は（num_examples、embedding_dimension）であることに注意。

In [0]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: id=400, shape=(3, 20), dtype=float32, numpy=
array([[ 3.9819887 , -4.4838037 ,  5.177359  , -2.3643482 , -3.2938678 ,
        -3.5364532 , -2.4786978 ,  2.5525482 ,  6.688532  , -2.3076782 ,
        -1.9807833 ,  1.1315885 , -3.0339816 , -0.7604128 , -5.743445  ,
         3.4242578 ,  4.790099  , -4.03061   , -5.992149  , -1.7297493 ],
       [ 3.4232912 , -4.230874  ,  4.1488533 , -0.29553518, -6.802391  ,
        -2.5163853 , -4.4002395 ,  1.905792  ,  4.7512794 , -0.40538004,
        -4.3401685 ,  1.0361497 ,  0.9744097 ,  0.71507156, -6.2657013 ,
         0.16533905,  4.560262  , -1.3106939 , -3.1121316 , -2.1338716 ],
       [ 3.8508697 , -5.003031  ,  4.8700504 , -0.04324996, -5.893603  ,
        -5.2983093 , -4.004676  ,  4.1236343 ,  6.267754  ,  0.11632943,
        -3.5934832 ,  0.8023905 ,  0.56146765,  0.9192484 , -7.3066816 ,
         2.8202746 ,  6.2000837 , -3.5709393 , -4.564525  , -2.305622  ]],
      dtype=float32)>

フルモデルをビルド

In [0]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


レイヤーは順番に積み重ねられて、分類が実行される

1. 最初のレイヤーはTensorFlow Hubレイヤー。このレイヤーは、事前学習済みの保存モデルを使用して、文を埋め込みベクトルにマッピングする。使用している事前学習済みのテキスト埋め込みモデル（google/tf2-preview/gnews-swivel-20dim/1）は、文をトークンに分割し、各トークンを埋め込み、埋め込みを結合する。結果の次元は次：（num_examples、embedding_dimension）。 
2. 2つ目のレイヤーは、この固定長の出力ベクトルは、16の非表示ユニットを持つ完全に接続された（密な）レイヤーにパイプされる。
3. 最後のレイヤーは、単一の出力ノードと密に接続されてる。シグモイド活性化関数を使用すると、この値は0〜1の浮動小数点数であり、確率または信頼レベルを表す。

#### モデルをコンパイルする

### 損失関数とオプティマイザ
モデルには、損失関数とトレーニング用のオプティマイザが必要。これはバイナリ分類問題であり、モデルは確率（シグモイド活性化を持つ単一ユニットレイヤー）を出力するため、binary_crossentropy損失関数を使用する。

損失関数の選択はこれだけではない。たとえば、mean_squared_errorを選択できる。ただし、一般に、binary_crossentropyは確率を扱うのに適している。これは、確率分布間の「距離」を測定する。

後で、回帰の問題を調べているとき（家の価格を予測するためなど）、平均二乗誤差と呼ばれる別の損失関数を使用をする。
次に、オプティマイザと損失関数を使用するようにモデルを構成する。

In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

### 学習とモデル
512サンプルのミニバッチで20エポックのモデルをトレーニングする。これは、x_trainテンソルとy_trainテンソルのすべてのサンプルで20回繰り返される。トレーニング中に、検証セットの10,000サンプルでモデルの損失と精度を監視する。

In [0]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 10s 335ms/step - loss: 1.6308 - accuracy: 0.4842 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/20
30/30 [==============================] - 9s 307ms/step - loss: 0.8751 - accuracy: 0.5173 - val_loss: 0.8030 - val_accuracy: 0.5373
Epoch 3/20
30/30 [==============================] - 9s 306ms/step - loss: 0.7726 - accuracy: 0.5554 - val_loss: 0.7426 - val_accuracy: 0.5713
Epoch 4/20
30/30 [==============================] - 10s 324ms/step - loss: 0.7162 - accuracy: 0.5906 - val_loss: 0.6999 - val_accuracy: 0.5977
Epoch 5/20
30/30 [==============================] - 9s 314ms/step - loss: 0.6758 - accuracy: 0.6205 - val_loss: 0.6652 - val_accuracy: 0.6309
Epoch 6/20
30/30 [==============================] - 9s 316ms/step - loss: 0.6391 - accuracy: 0.6478 - val_loss: 0.6362 - val_accuracy: 0.6581
Epoch 7/20
30/30 [==============================] - 10s 323ms/step - loss: 0.6101 - accuracy: 0.6733 - val_loss: 0.6097 - val_accuracy: 0.

### モデルを評価
そして、モデルのパフォーマンスを見てみる。 損失（エラーを表す数値、値が小さいほど良い）、および精度の2つの値が返される。

In [0]:
result = model.evaluate(test_data.batch(512), verbose=2)
for name, value in zip(model.metrics_names, result):
  print("%s: %.3f" % (name, value))

49/49 - 6s - loss: 0.3441 - accuracy: 0.8514
loss: 0.344
accuracy: 0.851


このかなり単純なアプローチにより、約85％の精度が達成される。より高度なアプローチでは、モデルは95％に近づくはず。